# Deep Learning assignment #1

***Student:*** [Isac Pasianotto](https://github.com/IsacPasianotto/dlhw)

In [ ]:
import torch
from typing import Iterable, List, Tuple
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset
from itertools import product
import matplotlib.pyplot as plt

In [ ]:
# Some constants
STD_DTYPE = torch.float32
NDIGITS = 6
N_HIDDEN = 2
N_HEPOCHS = 1425

## 0. Dataset definition

The considered dataset is the set of all possible 6-bit long binary sequences. Since the task the network is supposed to perform is to check if a given sequence is palindromic, the dataset is labeled accordingly.

In [ ]:
def is_symmetric(iterable: Iterable) -> float:
    """
    Check if an iterable is palindromic.
    :param iterable: an iterable object to check (in our case, a torch tensor)
    :return: 1 if the iterable is palindromic, 0 otherwise
    """
    return 1. if all(a == b for a, b in zip(iterable, reversed(iterable))) else 0.

# Ensure that the function works as expected
assert is_symmetric( torch.tensor([0, 1, 0, 1, 1, 0]) ) == 0.
assert is_symmetric( torch.tensor([0, 1, 0, 0, 1, 0]) ) == 1.

In [ ]:
def build_dataset(nbits: int = NDIGITS, dtype: torch.dtype = STD_DTYPE , unbalanced: bool = True, seed: int = 42 ) -> Tuple[Tensor, Tensor]:
    """
    Build the dataset of all possible binary sequences of length nbits and their corresponding labels.
    :param nbits: the length (integer) of the binary sequences. Default is 6.
    :param dtype: the data type of the tensors. Default is torch.float32.
    :param unbalanced: if True, the dataset will contain all the possible binary sequences of length nbits. If False, the palindromic sequences will be repeated more time to balance the dataset.
    :param seed: the seed for the random number generator. Default is 42, which is the answer to the Ultimate Question of Life, the Universe, and Everything.
    :return: Two tensors: the first one contains all the binary sequences, the second one contains the corresponding labels.
    """
    bin_seq: List[Tuple[int]] = list(product([0, 1], repeat=nbits))  # generate all possible binary sequences of length nbits
    sim_true: List[Tuple[int]] = [seq for seq in bin_seq if is_symmetric(seq)]
    sim_false: List[Tuple[int]] = [seq for seq in bin_seq if seq not in sim_true]

    sim_true: Tensor = torch.tensor(sim_true, dtype=dtype)
    sim_false: Tensor = torch.tensor(sim_false, dtype=dtype)

    if unbalanced:
        x_data: Tensor = torch.cat([sim_true, sim_false], dim=0)
        y_data: Tensor = torch.cat([torch.ones(sim_true.shape[0], dtype=dtype), torch.zeros(sim_false.shape[0], dtype=dtype)], dim=0)
    else:
        balance_ratio: int = sim_false.shape[0] // sim_true.shape[0]
        x_data: Tensor = torch.cat([sim_false, torch.cat([sim_true]*balance_ratio, dim=0)], dim=0)
        y_data: Tensor = torch.cat([torch.zeros(sim_false.shape[0], dtype=dtype), torch.ones(sim_true.shape[0]*balance_ratio, dtype=dtype)], dim=0)
        
    n_seq: int = x_data.shape[0]
    rndidx: Tensor[int] = torch.randperm(n_seq, generator=torch.Generator().manual_seed(seed))
    # Shuffle the dataset
    x_data = x_data[rndidx]
    y_data = y_data[rndidx]
    
    return x_data, y_data

x_data, y_data = build_dataset()
x_data_balanced, y_data_balanced = build_dataset(unbalanced=False)

## 1. Network definition

The considered paper is quite cryptic regarding all the needed information to reproduce exactly the experiment. From that I have evinced that:
- The input layer has 6 neurons, one for each bit of the input sequence.
- There is just 1 hidden layer with 2 neurons.
- The output layer has 1 neuron, which is the output of the network.

Hence, we can re-draw the *Fig. 1* of the paper with a more conventional representation as:  

<center><img src="./images/img-00.png" style="width: 35%;"/></center>

Moreover, there is specified that: 
- The initial weights are randomly drawn from a uniform distribution in the range [-0.3, 0.3].
- The gradient descent algorithm was not stochastic, all the dataset was used to compute the gradient.
- The loss function used (called "Error" in the paper) is $\frac{1}{2}\sum_{c}\sum_{j}{\left( y_{i,c} - d_{i, c} \right)^2}$
- The train lasted for 1,425 epochs.
- The learning rate was $\varepsilon = 0.1$.


What is not specified is: 

- The activation function used in the hidden layer. The only mentioned non-linear function in the paper is $y_i = \frac{1}{1+e^{-x_i}}$ which is the sigmoid function and is already implemented in [PyTorch](https://pytorch.org/docs/stable/generated/https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.htmll).
- The initialization of the bias terms. I will assume that they are initialized to 0.

In [ ]:
class paper_net(torch.nn.Module):
    """
    The network as described in the paper. Class which extends torch.nn.Module.
    """
    
    def __init__(self, input_size: int = NDIGITS, hidden_size: int = 2, func: torch.nn.Module = torch.nn.Sigmoid(), dtype: torch.dtype = STD_DTYPE):
        """
        Initialize the network object.
        :param input_size: the size of the input layer. Should be equal to te number of bits of the input sequence. Default is 6.
        :param hidden_size: the size of the hidden layer. Default is 2.
        :param func: the activation function to use in the hidden layer. Default is torch.nn.Sigmoid.
        :param dtype: the data type of the tensors. Default is torch.float32.
        """
        super(paper_net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size = 1
        self.act_fun = func
        self.dtype = dtype
        self.w_min = -0.3
        self.w_max = 0.3
        self.b = 0.
        
        self.hidden = torch.nn.Linear(self.input_size, self.hidden_size)
        self.output = torch.nn.Linear(self.hidden_size, self.out_size)
        
        with torch.no_grad():
            # Weights initialization
            torch.nn.init.uniform_(self.hidden.weight,self.w_min, self.w_max)
            torch.nn.init.uniform_(self.output.weight,self.w_min, self.w_max)
            # Bias initialization
            torch.nn.init.constant_(self.hidden.bias, self.b)
            torch.nn.init.constant_(self.output.bias, self.b)
    
    def forward(self, x: Tensor) -> Tensor:
        """
        Forward pass of the network.
        :param x: the input tensor.
        :return: the output tensor.
        """
        x = self.act_fun(self.hidden(x))
        x = self.act_fun(self.output(x))
        return x

In [ ]:
def loss_fn(y_pred: Tensor, y_true: Tensor, dtype: torch.dtype = STD_DTYPE) -> Tensor:
    """
    Compute the loss function.
    :param y_pred: the predicted values.
    :param y_true: the true values.
    :param dtype: the data type of the tensors. Default is torch.float32.
    :return: the loss value.
    """
    return .5 * torch.sum(torch.pow(y_pred - y_true, 2), dtype=dtype)

#### 1.1 Training the network, storing the weights and the losses

Since is required to inspect the weights of the network during the training, I will define a custom train function to store at each iteration all the data I need.

In [ ]:
# Set the batch size to the number of samples in the dataset to use the whole dataset at each iteration.
# This guarantees that GD is performed instead of SGD.
data: DataLoader = DataLoader(TensorDataset(x_data, y_data), batch_size=x_data.shape[0])
data_balanced: DataLoader = DataLoader(TensorDataset(x_data_balanced, y_data_balanced), batch_size=x_data_balanced.shape[0])

In [ ]:
def train_net(net: torch.nn.Module,
              data: DataLoader,
              hidden_layer_weights: Tensor,
              output_layer_weights: Tensor,
              accuracy: Tensor,
              losses: Tensor,
              n_epochs: int = N_HEPOCHS,
              lr: float = 0.1,
              optimizer: torch.optim.Optimizer = torch.optim.SGD,
              dtype: torch.dtype = STD_DTYPE,
              ): # -> None:
    """
    Train the network and stores the weights of the network, the losses and the accuracy in the given tensors.
    :param net: the network to train.
    :param data: the DataLoader object containing the dataset.
    :param hidden_layer_weights: the tensor to store the hidden layer weights.
    :param output_layer_weights: the tensor to store the output layer weights.
    :param accuracy: the tensor to store the accuracy.
    :param losses: the tensor to store the losses.
    :param n_epochs: the number of epochs. Default is 1425.
    :param lr: the learning rate. Default is 0.1.
    :param optimizer: the optimizer to use. Default is torch.optim.SGD.
    :param dtype: the data type of the tensors. Default is torch.float32.
    :return: Four tensors: the hidden layer weights, the output layer weights, the accuracy and the losses.
    """
    optimizer = optimizer(net.parameters(), lr=lr)
    net.train()
  
    for epoch in range(n_epochs):
        correct_predictions = 0        
        for x, y in data:
            optimizer.zero_grad()
            y_pred = net(x)
            l = loss_fn(y_pred, y, dtype=dtype)
            losses[epoch] = l.item()

            with torch.no_grad():
                pred: Tensor = torch.round(y_pred)
                acc: Tensor = torch.sum(pred == y, dtype=dtype) / y.shape[0]
                accuracy[epoch] = acc

            l.backward()
            optimizer.step()

        with torch.no_grad():
            hidden_layer_weights[epoch] = net.hidden.weight.data
            output_layer_weights[epoch] = net.output.weight.data

#### 1.2 Plotting the results

Before continuing with the experiment, I will define a function to plot the results of the training. This will be handy to compare the results of different variations of the network.

In [ ]:
def plot_all(h_lay: Tensor, o_lay: Tensor, acc: Tensor, loss: Tensor, title: str) -> None:
    """
    Make a 2x2 plot with the hidden layer weights, the output layer weights, the accuracy and the loss.
    :param h_lay: Tensor containing the hidden layer weights at each epoch.
    :param o_lay: Tensor containing the output layer weights at each epoch.
    :param acc: Tensor containing the accuracy at each epoch.
    :param loss: Tensor containing the loss at each epoch.
    :param title: String containing the title of the image generated
    :return: 
    """
    
    # constants
    epochs = range(len(acc))
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan']
    fig, axs = plt.subplots(2, 2, figsize=(18, 12))
    h_lay_00 = h_lay[:, 0, :] # first neuron of the hidden layer
    h_lay_01 = h_lay[:, 1, :] # second neuron of the hidden layer
    
    
    ## Top-left: Accuracy & Loss
    axs[0, 0].plot(epochs, acc, color=colors[0])
    axs[0, 0].set_xlabel('Epochs', fontsize=14, fontweight='bold')
    axs[0, 0].set_ylabel('Accuracy in %', color=colors[0], fontsize=14, fontweight='bold', labelpad=10)
    axs[0, 0].tick_params('y', colors=colors[0], labelsize=10)
    for label in axs[0, 0].get_yticklabels(): label.set_fontweight('bold')
    axs[0, 0].set_title('Accuracy and Loss over Epochs', fontsize=16, fontweight='bold')
    twinx = axs[0, 0].twinx()
    twinx.plot(epochs, loss, color=colors[1])
    twinx.set_ylabel('Loss', color=colors[1], fontsize=15, fontweight='bold', labelpad=10)
    twinx.tick_params('y', colors=colors[1], labelsize=10)
    for label in twinx.get_yticklabels(): label.set_fontweight('bold')

    ## Top-right: Hidden layer weights
    
    for i in range(NDIGITS):
        axs[0, 1].plot(epochs, h_lay_00[:, i], color=colors[i], label=f'Neuron {i}')
    axs[0, 1].set_xlabel('Epochs', fontsize=12, fontweight='bold')
    axs[0, 1].set_ylabel('Weights', fontsize=12, fontweight='bold', labelpad=10)
    axs[0, 1].set_title('Hidden layer weights over Epochs: 1', fontsize=16, fontweight='bold')
    axs[0, 1].legend(loc='upper right', fontsize=9)
    
    ## Bottom-left: Hidden layer weights
    
    for i in range(NDIGITS):
        axs[1, 0].plot(epochs, h_lay_01[:, i], color=colors[i], label=f'Neuron {i}')
    axs[1, 0].set_xlabel('Epochs', fontsize=12, fontweight='bold')
    axs[1, 0].set_ylabel('Weights', fontsize=12, fontweight='bold', labelpad=10)
    axs[1, 0].set_title('Hidden layer weights over Epochs: 2', fontsize=16, fontweight='bold')
    axs[1, 0].legend(loc='upper right', fontsize=9)
    
    ## Bottom-right: Output layer weights
    for i in range(N_HIDDEN):
        axs[1, 1].plot(epochs, o_lay[:, 0, i], color=colors[i], label=f'Neuron {i}')
    axs[1, 1].set_xlabel('Epochs', fontsize=12, fontweight='bold')
    axs[1, 1].set_ylabel('Weights', fontsize=12, fontweight='bold', labelpad=10)
    axs[1, 1].set_title('Output layer weights over Epochs', fontsize=16, fontweight='bold')
    axs[1, 1].legend(loc='upper right', fontsize=9)
    
    fig.suptitle(title, fontsize=20, fontweight='bold')
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.3, hspace=0.3)
    plt.show()

## 2 Perform the experiment

#### 2.1 Original formulation

We will use the `unbalanced dataset` to train the network:

In [ ]:
model_00: paper_net = paper_net()
h_lay_00: Tensor = torch.zeros(N_HEPOCHS, N_HIDDEN, NDIGITS, dtype=STD_DTYPE)
o_lay_00: Tensor = torch.zeros(N_HEPOCHS, 1, N_HIDDEN, dtype=STD_DTYPE)
acc_00: Tensor = torch.zeros(N_HEPOCHS, dtype=STD_DTYPE)
loss_00: Tensor = torch.zeros(N_HEPOCHS, dtype=STD_DTYPE)

train_net(model_00, data, h_lay_00, o_lay_00, acc_00, loss_00)
plot_all(h_lay_00, o_lay_00, acc_00, loss_00, "Original formulation")

#### 2.2 Balanced dataset

Use the same network but with the `balanced dataset`:

In [ ]:
model_01: paper_net = paper_net()
h_lay_01: Tensor = torch.zeros(N_HEPOCHS, N_HIDDEN, NDIGITS, dtype=STD_DTYPE)
o_lay_01: Tensor = torch.zeros(N_HEPOCHS, 1, N_HIDDEN, dtype=STD_DTYPE)
acc_01: Tensor = torch.zeros(N_HEPOCHS, dtype=STD_DTYPE)
loss_01: Tensor = torch.zeros(N_HEPOCHS, dtype=STD_DTYPE)

train_net(model_01, data_balanced, h_lay_01, o_lay_01, acc_01, loss_01)
plot_all(h_lay_01, o_lay_01, acc_01, loss_01, "Balanced dataset")